# 1. Get miRNAs that are in (+) strand

In [ ]:
# get all mRNA gene names from gencode.gtf
!grep -i protein_coding ../binfo1-datapack1/gencode.gtf | awk '$3=="gene" && $7=="+" {print $10, $1, $4, $5}' > mirna_dm.txt

miRNA 중 (+) strand인 entity 들만 선정

In [ ]:
import pandas as pd
import re
from collections import Counter
from scipy.stats import entropy
from tqdm import tqdm
from datetime import datetime
import pickle

import parmap

model_name = 'get_Lin28a_binding_seqs'
version = 'parmap'
logger_file = f'log/{model_name}_v{version}'


def logger(text, show=True):
    text = f'[{datetime.now().replace(microsecond=0)}] {text}'
    with open(f'{logger_file}.log', 'a') as f:
        f.write(text+'\n')
    if show:
        print(text)

In [ ]:
mrnas = pd.read_csv('mirna_dm.txt', delimiter=' ', header=None)
mrnas.columns = ['gene_id','chr','start','end']
mrnas[mrnas['chr']!='chrM']
mrnas

In [ ]:
mrnas['gene_id'] = mrnas['gene_id'].str.split('.',expand=True)
mrnas # all gene ids are unique

In [ ]:
def count_base(seq):
    counts = Counter(seq.upper())
    counts = [counts['A'], counts['T'], counts['G'], counts['C']]
    index = counts.index(max(counts))
    return counts, index

In [ ]:
threshold_entropy = 1.2

genes = []
seqs = []


#for i in tqdm(range(0,100)):
    
def function(i):    
    gene = mrnas.iloc[i]
    chr = gene['chr']
    start = gene['start']
    end = gene['end']
    gid = gene['gene_id']
    
    query = f'{chr}:{start}-{end}'
    file = f'parmap/gene_{i}.pileup'

    !samtools view -b -o - ../binfo1-datapack1/CLIP-35L33G.bam $query | samtools mpileup - > $file
    pileup = pd.read_csv(file, sep='\t', names=['chrom', 'pos', '_ref', 'count', 'basereads', 'quals'])
    pileup = pileup[(pileup['pos']>=start)&(pileup['pos']<=end)]

    toremove = re.compile('[<>$*#^]')
    pileup['matches'] = pileup['basereads'].apply(lambda x: toremove.sub('', x))

    max_entropy = 0
    max_pos = 0
    tot_pos = []
    tot_seq = []
    for pos in pileup['pos']:
        seq = pileup[pileup['pos'] == pos].iloc[0]['matches']

        # if seq is null : insert entropy as 1
        if seq != '' : 
            count_list, index = count_base(seq)
            entrop = entropy(count_list)
            tot_seq.append(index)
            tot_pos.append(pos)
            #print(pos, entrop, count_base(seq))

            if max_entropy < entrop:
                max_entropy = entrop
                max_pos = pos
                
    try: 
        max_count = pileup[pileup['pos']==max_pos].iloc[0]['count']
    except:
        max_count = 0
    
    if max_entropy > threshold_entropy:# & (max_count >= 50): # generate sequences
        tot_seq[tot_pos.index(max_pos)]=2 # attatched location = G(2)
        try:
            seq_st = tot_pos.index(max_pos-8)
            seq_end = tot_pos.index(max_pos+9)
            seq18 = tot_seq[seq_st:seq_end+1]
            genes.append(gid)
            seqs.append(seq18)   
            logger(f'[{max_entropy}/{max_count}] {gid} is added!')
        except: 
            try:
                seq_st = tot_pos.index(max_pos-8)
                seq18 = tot_seq[seq_st:]
                genes.append(gid)
                seqs.append(seq18)   
                logger(f'[{max_entropy}/{max_count}] {gid} is added!')
            except:
                try:
                    seq_end = tot_pos.index(max_pos+9)
                    seq18 = tot_seq[:seq_end+1]
                    genes.append(gid)
                    seqs.append(seq18)   
                    logger(f'[{max_entropy}/{max_count}] {gid} is added!')
                except:
                    pass
                
    with open(f'parmap/seqs_mrna_{i}.pkl', 'wb') as f:
        pickle.dump((genes, seqs), f)

In [ ]:
# with open('seqs_mrna.pkl', 'wb') as f:
#     pickle.dump((genes, seqs), f)
    
# with open('seqs_mrna.pkl', 'rb') as f:
#     a,b = pickle.load(f)
    
# len(a)

In [ ]:
# parmap.map(function, range(0,5000), pm_pbar=True, pm_processes=120)

In [ ]:
parmap.map(function, range(5000,len(mrnas)), pm_pbar=True, pm_processes=120)